In [17]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Dropout
from keras.models import Model
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [18]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [19]:
print("Train Images Shape : " + str(train_images.shape))
print("Test Images Shape : " + str(test_images.shape))
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

Train Images Shape : (50000, 32, 32, 3)
Test Images Shape : (10000, 32, 32, 3)


In [20]:
# The classes in CIFAR-10 dataset
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [21]:
vgg19 = VGG19(weights="imagenet", include_top=False, input_shape=(32,32, 3))
vgg19.trainable = False

In [22]:
x = vgg19.output
x = GlobalAveragePooling2D()(x)

x = BatchNormalization()(x)

x = Dropout(0.5)(x) 
x = Dense(512, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(10, activation ='softmax')(x)


model = Model(vgg19.input, x)
model.compile(optimizer ='Adam', 
              loss ="sparse_categorical_crossentropy", 
              metrics =["sparse_categorical_accuracy"]) 

history = model.fit(train_images,train_labels, epochs = 5, validation_data = (test_images,test_labels), verbose = 1, shuffle=True)
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy: {}".format(test_accuracy))

Epoch 1/5
1563/1563 [==============================] - 827s 529ms/step - loss: 1.8048 - sparse_categorical_accuracy: 0.4023 - val_loss: 1.3744 - val_sparse_categorical_accuracy: 0.5200
Epoch 2/5
1563/1563 [==============================] - 810s 518ms/step - loss: 1.5588 - sparse_categorical_accuracy: 0.4482 - val_loss: 1.3261 - val_sparse_categorical_accuracy: 0.5284
Epoch 3/5
1563/1563 [==============================] - 824s 527ms/step - loss: 1.5261 - sparse_categorical_accuracy: 0.4638 - val_loss: 1.3064 - val_sparse_categorical_accuracy: 0.5354
Epoch 4/5
1563/1563 [==============================] - 826s 529ms/step - loss: 1.4984 - sparse_categorical_accuracy: 0.4711 - val_loss: 1.2900 - val_sparse_categorical_accuracy: 0.5459
Epoch 5/5
313/313 [==============================] - 137s 439ms/step - loss: 1.2758 - sparse_categorical_accuracy: 0.5546
Test accuracy: 0.5546000003814697


In [24]:
model.save('transfer_model.h5')

In [ ]:
"""
Run this cell, by replacing "<IMAGE_PATH_HERE>" with your image path to test a custom image
"""
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('transfer_model.h5')

def test(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (32, 32))
    img = img / 255.0
    res = model.predict(np.array([img]))
    ans = np.argmax(res)
    print(class_names[ans])

test("<IMAGE_PATH_HERE>")